# Introduction

Prompt templates are predefined recipes for generating prompts for language models.

A template may include instructions, few-shot examples, and specific context and questions appropriate for a given task.

LangChain provides tooling to create and work with prompt templates.

LangChain strives to create model agnostic templates to make it easy to reuse existing templates across different language models.

Typically, language models expect the prompt to either be a string or else a list of chat messages.

# PromptTemplates

https://python.langchain.com/docs/how_to/installation/

https://www.mirascope.com/post/langchain-prompt-template

https://www.mirascope.com/post/prompt-engineering-tools

https://python.langchain.com/docs/how_to/#prompt-templates

A LangChain prompt template is a class containing elements you typically need for a Large Language Model (LLM) prompt. At a minimum, these are:

A natural language string that will serve as the prompt: This can be a simple text string, or, for prompts consisting of dynamic content, an f-string or docstring containing placeholders that represent variables.
‍
Formatting instructions (optional), that specify how dynamic content should appear in the prompt, i.e., whether it should be italicized, capitalized, etc. 
‍
Input parameters (optional) that you pass into the prompt class to provide instructions or context for generating prompts. These parameters influence the content, structure, or formatting of the prompt. But oftentimes they’re variables for the placeholders in the string, whose values resolve to produce the final string that goes into the LLM through an API call as the prompt.

In [1]:
%run utilities.ipynb
llm_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) 

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


## `PromptTemplate` for creating basic prompts.

It contains all the elements needed to create the prompt, but doesn’t feature autocomplete for the variables `dish` and `flavor`.

LangChain’s templates use Python’s `str.format` by default, but for complex prompts you can also use jinja2. 

In [2]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
  "Write a delicious recipe for {dish} with a {flavor} twist."
)

# Formatting the prompt with new content
formatted_prompt = prompt_template.format(dish="pasta", flavor="spicy")

print(formatted_prompt)

Write a delicious recipe for pasta with a spicy twist.


In [3]:
#USe it with an LLM

answer=llm_model.invoke(formatted_prompt)

In [4]:
print(answer)

content='Spicy Arrabbiata Pasta\n\nIngredients:\n- 1 pound of penne pasta\n- 2 tablespoons of olive oil\n- 4 cloves of garlic, minced\n- 1 teaspoon of red pepper flakes\n- 1 can of crushed tomatoes\n- 1 teaspoon of sugar\n- Salt and pepper to taste\n- Fresh basil leaves, chopped\n- Grated Parmesan cheese for serving\n\nInstructions:\n1. Cook the penne pasta according to package instructions until al dente. Drain and set aside.\n2. In a large skillet, heat the olive oil over medium heat. Add the minced garlic and red pepper flakes, and sauté for 1-2 minutes until fragrant.\n3. Add the crushed tomatoes and sugar to the skillet, stirring to combine. Season with salt and pepper to taste.\n4. Simmer the sauce for 10-15 minutes, allowing the flavors to meld together and the sauce to thicken slightly.\n5. Add the cooked penne pasta to the skillet, tossing to coat the pasta in the spicy arrabbiata sauce.\n6. Serve the spicy arrabbiata pasta hot, garnished with fresh basil leaves and grated Par

In [5]:
print(answer.content)

Spicy Arrabbiata Pasta

Ingredients:
- 1 pound of penne pasta
- 2 tablespoons of olive oil
- 4 cloves of garlic, minced
- 1 teaspoon of red pepper flakes
- 1 can of crushed tomatoes
- 1 teaspoon of sugar
- Salt and pepper to taste
- Fresh basil leaves, chopped
- Grated Parmesan cheese for serving

Instructions:
1. Cook the penne pasta according to package instructions until al dente. Drain and set aside.
2. In a large skillet, heat the olive oil over medium heat. Add the minced garlic and red pepper flakes, and sauté for 1-2 minutes until fragrant.
3. Add the crushed tomatoes and sugar to the skillet, stirring to combine. Season with salt and pepper to taste.
4. Simmer the sauce for 10-15 minutes, allowing the flavors to meld together and the sauce to thicken slightly.
5. Add the cooked penne pasta to the skillet, tossing to coat the pasta in the spicy arrabbiata sauce.
6. Serve the spicy arrabbiata pasta hot, garnished with fresh basil leaves and grated Parmesan cheese.
7. Enjoy the d

## Try your own

In [6]:

question="Find two opposite perspectives for the query: {query}"
prompt_template = PromptTemplate.from_template(question)
# Formatting the prompt with new content

query="Who overthrew the government in Ukraine in 2014"
formatted_prompt = prompt_template.format(query=query)

print(formatted_prompt)
answer=llm_model.invoke(formatted_prompt)

Find two opposite perspectives for the query: Who overthrew the government in Ukraine in 2014


In [7]:
print(answer.content)

Perspective 1: The Ukrainian people, led by pro-democracy activists and opposition leaders, overthrew the government in 2014 through a popular uprising known as the Euromaidan revolution. They were fed up with the corrupt and authoritarian regime of President Viktor Yanukovych and demanded his resignation.

Perspective 2: The government of Ukraine was overthrown in 2014 through a violent coup orchestrated by Western powers, particularly the United States and the European Union. They supported and funded opposition groups to destabilize the country and install a pro-Western government that would serve their interests.


# `FewShotPromptTemplate`

Often a more useful form of prompting than sending a simple string with a request or question is to include several examples of outputs you want.

This is few-shot learning and is used to train models to do new tasks well, even when they have only a limited amount of training data available.

Many real-world use cases benefit from few-shot learning, for instance:

An automated fact checking tool where you provide different few-shot examples where the model is shown how to verify information, ask follow-up questions if necessary, and conclude whether a statement is true or false.
‍
A technical support and troubleshooting guide that assists users in diagnosing and solving issues with products or software, where the `FewShotPromptTemplate` could contain examples of common troubleshooting steps, including how to ask the user for specific system details, interpret symptoms, and guide them through the solution process.

The `FewShotPromptTemplate` class takes a list of (question-and-answer) dictionaries as input, before asking a new question:

In [8]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
    {
        "question": "What is the tallest mountain in the world?",
        "answer": "Mount Everest",
    },
    {"question": "What is the largest ocean on Earth?", "answer": "Pacific Ocean"},
    {"question": "In which year did the first airplane fly?", "answer": "1903"},
]

example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\n{answer}",
)
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

prompt=prompt_template.format(
        input="What is the name of the famous clock tower in London?"
    )
print(prompt)


Question: What is the tallest mountain in the world?
Mount Everest

Question: What is the largest ocean on Earth?
Pacific Ocean

Question: In which year did the first airplane fly?
1903

Question: What is the name of the famous clock tower in London?


In [9]:
answer=llm_model.invoke(prompt)

In [10]:
print(answer)

content='Big Ben' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 62, 'total_tokens': 64, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-630d089d-823a-4ef6-bf86-80f3ed609984-0' usage_metadata={'input_tokens': 62, 'output_tokens': 2, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}


In [11]:
print(answer.content)

Big Ben


## Try your own

In [12]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# Step 1: Define the few-shot examples
examples = [
    {
        "input": "Explain quantum computing to high school students",
        "output": (
            "Quantum computing is a new type of computing that uses the strange and fascinating rules of quantum "
            "mechanics, the science that explains how the smallest particles in the universe behave. Unlike regular "
            "computers, which use bits (0s and 1s) to process information, quantum computers use qubits. Qubits can "
            "be both 0 and 1 at the same time, thanks to a property called superposition. This means quantum computers "
            "can solve certain problems much faster than classical computers. Imagine being able to solve really "
            "complex math problems or help scientists find cures for diseases in seconds instead of years—this is the "
            "promise of quantum computing!"
        )
    },
    {
        "input": "Introduce quantum computing in a simple way to high school students",
        "output": (
            "Have you ever wondered how computers work? They process information using tiny switches that can either "
            "be 'on' or 'off'—this is called binary. Quantum computers, on the other hand, use something even more "
            "amazing called 'qubits,' which can be 'on,' 'off,' or both at the same time! This allows quantum computers "
            "to perform many calculations all at once. Although they're still being developed, quantum computers could "
            "change fields like medicine, chemistry, and even help design new technologies by solving problems that are "
            "way too complicated for today's computers."
        )
    },
    {
        "input": "Describe quantum computing for teenagers",
        "output": (
            "Think of a regular computer like a really fast calculator that can only do one thing at a time. Quantum "
            "computers, however, can do many things at once! They use principles from quantum physics, a field that "
            "deals with the tiniest particles, like atoms. One important idea is called 'superposition,' which allows "
            "a quantum bit (or qubit) to be in multiple states at once, making it super powerful for solving big "
            "problems like breaking secret codes, simulating molecules, or even creating better artificial "
            "intelligence. While it’s still new, quantum computing could lead to breakthroughs we can't even imagine yet!"
        )
    },
    {
        "input": "Explain what quantum computing is to students in a fun way",
        "output": (
            "Imagine you have a regular computer that’s like a detective solving a mystery—one clue at a time. A "
            "quantum computer is like having hundreds of detectives working together, checking different clues all at "
            "once! This is because quantum computers use 'qubits,' which can be in multiple states at the same time. "
            "This makes them super fast at solving certain kinds of problems that are tough for normal computers. For "
            "example, quantum computers could help design new drugs, improve cybersecurity, or even simulate how planets "
            "move in space. Pretty cool, right?"
        )
    },
    {
        "input": "Summarize quantum computing in a way that’s easy to understand for high school students",
        "output": (
            "Quantum computing is like upgrading your brainpower! A regular computer uses bits that are either 0 or 1 "
            "to do its calculations. But quantum computers use qubits, which can be both 0 and 1 at the same time. This "
            "allows them to work on many tasks simultaneously, making them much faster for certain kinds of problems. "
            "For example, they could help scientists make new medicines or even design materials that don't exist yet. "
            "The world of quantum computing is still being explored, but it's expected to change the way we solve some "
            "of the hardest problems in science and technology."
        )
    }
]


# Step 2: Create a PromptTemplate to structure the input format
example_prompt = PromptTemplate(
    input_variables=["input"],
    template="{input}"
)

# Step 3: Build the FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,                # The few-shot examples defined above
    example_prompt=example_prompt,     # How to structure each example
    suffix="Input: {input}\nOutput:",  # How the final prompt will appear
    input_variables=["input"]          # The variable that will be provided by the user
)

# Step 4: Test it by passing a new input to the few_shot_prompt

new_input = {"input": "Show an example how Simon's algorithm in quantum computing is used"}
prompt = few_shot_prompt.format(input=new_input["input"])

# Print the generated final prompt
print(prompt)


Explain quantum computing to high school students

Introduce quantum computing in a simple way to high school students

Describe quantum computing for teenagers

Explain what quantum computing is to students in a fun way

Summarize quantum computing in a way that’s easy to understand for high school students

Input: Show an example how Simon's algorithm in quantum computing is used
Output:


In [13]:
answer=llm_model.invoke(prompt)
print(answer.content)

Imagine you have a super powerful computer that can solve complex problems much faster than any regular computer. This is what quantum computing is all about. Instead of using regular bits like in traditional computers, quantum computers use quantum bits or qubits. These qubits can exist in multiple states at the same time, allowing them to process a huge amount of information simultaneously. 

Now, let's take a look at Simon's algorithm in quantum computing. This algorithm is used to find patterns in data much faster than classical computers. For example, if you have a list of numbers and you want to find a hidden pattern in them, Simon's algorithm can do this in a fraction of the time it would take a regular computer. This is just one of the many amazing things quantum computing can do!


In [14]:
examples = [
    {   "input":"Create a function that adds two numbers.",
         "output": (
                    "def add_numbers(a, b):"
                        "return a + b"
         )
   }]
    
# Step 2: Create a PromptTemplate to structure the input format
example_prompt = PromptTemplate(
    input_variables=["input"],
    template="{input}"
)

# Step 3: Build the FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,                # The few-shot examples defined above
    example_prompt=example_prompt,     # How to structure each example
    suffix="Input: {input}\nOutput:",  # How the final prompt will appear
    input_variables=["input"]          # The variable that will be provided by the user
)

# Step 4: Test it by passing a new input to the few_shot_prompt
new_input = {"input": "Create a code snippet that multiplies two given numbers"}
prompt = few_shot_prompt.format(input=new_input["input"])

# Print the generated final prompt
print(prompt)
answer=llm_model.invoke(prompt)   

Create a function that adds two numbers.

Input: Create a code snippet that multiplies two given numbers
Output:


In [15]:
print(answer.content)

```python
def multiply_numbers(num1, num2):
    return num1 * num2

result = multiply_numbers(5, 10)
print(result)  # Output: 50
```


In [16]:
examples = [
    {   "question": "Who overthrew the government in Iran?",
        "answer": "The US and British governments",
    },
    {"question": "Why did they overthrow the government in Iran?", 
     "answer": "To control the oil"},
    {"question": "How does the US and Britain overthrow governments?", 
     "answer": "They stage coup d'etats and use proxies"},
]

example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\n{answer}",
)
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

query="Why was the government in Ukraine overthrown in 2014"
question=f"""Find two opposite perspectives for the query: {query}"""

prompt=prompt_template.format(
        input=question
    )
print(prompt)

Question: Who overthrew the government in Iran?
The US and British governments

Question: Why did they overthrow the government in Iran?
To control the oil

Question: How does the US and Britain overthrow governments?
They stage coup d'etats and use proxies

Question: Find two opposite perspectives for the query: Why was the government in Ukraine overthrown in 2014


In [17]:
answer=llm_model.invoke(prompt)

In [18]:
print(answer.content)

Perspective 1: The government in Ukraine was overthrown in 2014 due to widespread corruption, economic instability, and a lack of democratic reforms. The people of Ukraine were fed up with the government's inability to address their needs and took to the streets in protest, ultimately leading to the ousting of President Yanukovych.

Perspective 2: The government in Ukraine was overthrown in 2014 as part of a Western-backed coup orchestrated to further NATO's expansion and weaken Russia's influence in the region. The United States and European Union supported opposition groups in Ukraine, leading to the removal of President Yanukovych and the installation of a pro-Western government.


In [19]:
examples = [
    {"query": "Start of LaTeX document.",
     "output":
        "\\documentclass{{article}}"
        "\\usepackage{{amsmath}}"
        "\\usepackage[a4paper, margin=0.5in]{{geometry}}"  
    },   
        
    {"query":"Matrices",
     "output":  
               "\["
                "\begin{{pmatrix}}"
                "1 & 2 & 3 \\"
                "4 & 5 & 6 \\"
                "7 & 8 & 9"
                "\end{{pmatrix}}"
                "\]"
    } ,

    {"query": "Inline Math",
     "output" : "$ ... $"}
    ,
                
    {"query": "Displayed Equations",
     "output": " \[ ... \] "},
                
    {"query": "Sections and Items",
     "output": "\section*{{}} for unnumbered sections and enumerate or itemize for lists."}
   ]

example_prompt = PromptTemplate(
    input_variables=["query", "output"],
    template="Question: {query}\n{output}",
)
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)





### First create a problem. The output will be turned into a Latex document

In [20]:
prompt = PromptTemplate.from_template(
  "Generate a physics problem that involves matrices and eigenvalues. Provide a step by step solution"
)
prompt=prompt.format()
physics_problem1=llm_model.invoke(prompt)
print(physics_problem1.content)

Problem: A 2x2 matrix A represents a linear transformation in a certain coordinate system. The matrix A has eigenvalues of 3 and -1. Find the matrix A and the corresponding eigenvectors.

Solution:
Step 1: The characteristic equation of a 2x2 matrix A is given by det(A - λI) = 0, where λ is the eigenvalue and I is the identity matrix. Since the eigenvalues are 3 and -1, the characteristic equation becomes:
det(A - 3I) = 0
det(A + I) = 0

Step 2: Solve for the matrix A by setting up the characteristic equations:
(A - 3I)(A + I) = 0
A^2 - 2A - 3I = 0
A^2 - 2A - 3I = 0

Step 3: Solve for the matrix A by finding the eigenvectors corresponding to the eigenvalues 3 and -1. For the eigenvalue 3:
(A - 3I)v = 0
(A - 3I)v = 0
(A - 3I)v = 0

Step 4: Solve for the eigenvector v by setting up the equation:
(A - 3I)v = 0
(A - 3I)v = 0
(A - 3I)v = 0

Step 5: Solve for the eigenvector v by finding the null space of the matrix A - 3I:
(A - 3I)v = 0
(A - 3I)v = 0
(A - 3I)v = 0

Step 6: Repeat steps 3-5 

In [21]:
query=f""""Generate a latex document for the following input: {physics_problem1.content}"""

prompt=few_shot_prompt_template.format(
        input=query
    )
latex_answer1=llm_model.invoke(prompt)
print(latex_answer1.content)

\documentclass{article}
\usepackage{amsmath}
\usepackage[a4paper, margin=0.5in]{geometry}

\begin{document}

\section*{Problem}
A 2x2 matrix $A$ represents a linear transformation in a certain coordinate system. The matrix $A$ has eigenvalues of 3 and -1. Find the matrix $A$ and the corresponding eigenvectors.

\section*{Solution}
\textbf{Step 1:} The characteristic equation of a 2x2 matrix $A$ is given by $\text{det}(A - \lambda I) = 0$, where $\lambda$ is the eigenvalue and $I$ is the identity matrix. Since the eigenvalues are 3 and -1, the characteristic equation becomes:
\[\text{det}(A - 3I) = 0\]
\[\text{det}(A + I) = 0\]

\textbf{Step 2:} Solve for the matrix $A$ by setting up the characteristic equations:
\[(A - 3I)(A + I) = 0\]
\[A^2 - 2A - 3I = 0\]
\[A^2 - 2A - 3I = 0\]

\textbf{Step 3:} Solve for the matrix $A$ by finding the eigenvectors corresponding to the eigenvalues 3 and -1. For the eigenvalue 3:
\[(A - 3I)v = 0\]
\[(A - 3I)v = 0\]
\[(A - 3I)v = 0\]

\textbf{Step 4:} So

In [36]:
def Save_File(inp,name:str): 
     
    file_path = fr"./{name}.tex"
    with open(file_path, "w",encoding="utf-8") as file:
        file.write(inp)
    
Save_File(physics_problem1.content,"latex1_file")

# `ChatPromptTemplate`

The `ChatPromptTemplate` class focuses on the conversation flow between a user and an AI system, and provides instructions or requests for roles like user, system, assistant, and others (the exact roles you can use will depend on your LLM model).

Such roles give deeper context to the LLM and elicit better responses that help the model grasp the situation more holistically. System messages in particular provide implicit instructions or set the scene, informing the LLM of expected behavior.

The roles in this class are:

`System` for a system chat message setting the stage (e.g., “You are a knowledgeable historian”).
‍
`User`, which contains the user’s specific historical question.
‍
`AI`, which contains the LLM’s preliminary response or follow-up question.
Once the template object is instantiated, you can use it to generate chat prompts by replacing the placeholders with actual content.

This prompt sets the context, provides the user's question, and typically leaves the AI response blank for the LLM to generate.

In [23]:
from langchain_core.prompts import ChatPromptTemplate

# Define roles and placeholders
chat_template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a knowledgeable AI assistant. You are called {name}."),
    ("user", "Hi, what's the weather like today?"),
    ("ai", "It's sunny and warm outside."),
    ("user", "{user_input}"),
   ]
)

messages = chat_template.format_messages(name="Alice", user_input="Can you tell me a joke?")

In [24]:
print(messages)

[SystemMessage(content='You are a knowledgeable AI assistant. You are called Alice.', additional_kwargs={}, response_metadata={}), HumanMessage(content="Hi, what's the weather like today?", additional_kwargs={}, response_metadata={}), AIMessage(content="It's sunny and warm outside.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you tell me a joke?', additional_kwargs={}, response_metadata={})]


In [25]:
answer=llm_model.invoke(messages)

In [26]:
print(answer)

content="Sure! Why couldn't the bicycle find its way home? Because it lost its bearings!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 54, 'total_tokens': 72, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-171d5493-9724-4216-a555-c512dbbd9fa0-0' usage_metadata={'input_tokens': 54, 'output_tokens': 18, 'total_tokens': 72, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}


In [27]:
print(answer.content)

Sure! Why couldn't the bicycle find its way home? Because it lost its bearings!


## Try your own

In [28]:
class Formatting():
    
    def Latex_math(self):
        return f""" 
        - Use  
        \\documentclass{{article}}
        \\usepackage{{amsmath}}
        \\usepackage[a4paper, margin=0.5in]{{geometry}}  
        
        -If you use matrices use the following example as a guide:
                ***Example***:
               "\[
                \begin{{pmatrix}}
                1 & 2 & 3 \\
                4 & 5 & 6 \\
                7 & 8 & 9
                \end{{pmatrix}}
                \]"
        - Inline Math: Use $ ... $ for inline math.
                
        - Displayed Equations: Use \[ ... \] for displayed equations, which centers the math on its own line.
                
        - Sections and Items: Use \section*{{}} for unnumbered sections and enumerate or itemize for lists.

        - Always ensure that any opened brackets—such as `{{}}`, `[]`, or `()`—are properly closed in your expressions.

        -  Paragraph Formatting**: Break the text into paragraphs appropriately. 
            Use LaTeX commands to manage text justification to prevent overfull `\hbox` issues.
        """
        
    def Latex_exam(self):
            return f""" - Use this example as a guide 
            \section*{{Section A: Multiple-Choice-Questions ( points)}}
            \begin{{enumerate}}
                \item (2 points)
                 A car accelerates uniformly from rest to a speed of 20 m/s in 5 seconds. What is the car's acceleration?
                \begin{{itemize}}
                    \item[i)] 2 m/s²
                    \item[ii)] 4 m/s²
                    \item[iii)] 5 m/s²
                    \item[iv)] 10 m/s²
                \end{{itemize}}
            \end{{enumerate}}
            """
       
        
    def Latex_Formatting(self,application,text):
        L_example=""
        if application=="exam":
            L_example=self.Latex_exam()
        L_format=self.Latex_math()
        SYS_PROMPT = fr"""You are a highly skilled LaTeX professional. 
            Your role is to take a user-provided content, which may be in any language, 
            and convert it into a formatted, well-structured LaTeX document. 
            Adhere strictly to the specified formatting guidelines to ensure the document is professional, clean, and suitable for formal use."""
    
    
     
        USER_PROMPT = fr"""
              You have been provided with the following content: {text}.
              Please format the provided content into a LaTeX document, adhering strictly to the following formatting guidelines.\n
            1. Preserve the content exactly as it is, without making any changes, omissions, or rearrangements, regardless of the language.
            2. Format each section individually, maintaining a clear and logical structure.
            3. Maintain the original order of the content throughout the document.
            4. Apply the following specific formatting instructions: {L_format}.
            {L_example}
            5.Ensure the resulting LaTeX document is well-organized, professionally formatted, and ready for formal presentation.
           
            """

        
        messages = [
            SystemMessage(content=SYS_PROMPT), 
            HumanMessage(content=USER_PROMPT)  
        ]
 
        return messages



### First create a physics problem that afterwards will be converted into a Latex document

In [29]:
prompt1 = PromptTemplate.from_template(
  "Generate a physics problem that involves matrices and eigenvalues. Provide a step by step solution"
)
prompt1=prompt1.format()
physics_problem=llm_model.invoke(prompt1)


In [30]:
latex=Formatting()

In [31]:
prompt2=latex.Latex_Formatting("",physics_problem.content)

In [32]:
print(prompt2)

[SystemMessage(content='You are a highly skilled LaTeX professional. \n            Your role is to take a user-provided content, which may be in any language, \n            and convert it into a formatted, well-structured LaTeX document. \n            Adhere strictly to the specified formatting guidelines to ensure the document is professional, clean, and suitable for formal use.', additional_kwargs={}, response_metadata={}), HumanMessage(content='\n              You have been provided with the following content: Problem: A 2x2 matrix A represents a linear transformation in a certain coordinate system. Find the eigenvalues and eigenvectors of matrix A.\n\nMatrix A = [[3, 1],\n            [1, 3]]\n\nSolution:\n1. To find the eigenvalues, we need to solve the characteristic equation det(A - λI) = 0, where λ is the eigenvalue and I is the identity matrix.\n\ndet([[3-λ, 1],\n     [1, 3-λ]]) = 0\n\n(3-λ)(3-λ) - 1 = 0\nλ^2 - 6λ + 8 = 0\n(λ-4)(λ-2) = 0\n\nTherefore, the eigenvalues are λ1 = 4

In [33]:
answer=llm_model.invoke(prompt2)

In [34]:
print(answer.content)

```latex
\documentclass{article}
\usepackage{amsmath}
\usepackage[a4paper, margin=0.5in]{geometry}

\begin{document}

\section*{Problem}
A $2\times2$ matrix $A$ represents a linear transformation in a certain coordinate system. Find the eigenvalues and eigenvectors of matrix $A$.

Matrix $A = \begin{pmatrix}
3 & 1 \\
1 & 3
\end{pmatrix}$

\section*{Solution}
1. To find the eigenvalues, we need to solve the characteristic equation $\text{det}(A - \lambda I) = 0$, where $\lambda$ is the eigenvalue and $I$ is the identity matrix.

\[
\text{det}\left(\begin{pmatrix}
3-\lambda & 1 \\
1 & 3-\lambda
\end{pmatrix}\right) = 0
\]

\[
(3-\lambda)(3-\lambda) - 1 = 0
\]
\[
\lambda^2 - 6\lambda + 8 = 0
\]
\[
(\lambda-4)(\lambda-2) = 0
\]

Therefore, the eigenvalues are $\lambda_1 = 4$ and $\lambda_2 = 2$.

2. To find the eigenvectors, we substitute each eigenvalue back into the equation $(A - \lambda I)v = 0$ and solve for $v$.

For $\lambda_1 = 4$:
\[
(A - 4I)v = 0
\]
\[
\begin{pmatrix}
-1 & 1 \\
1

In [35]:
Save_File(answer.content,"latex2_file")